In [13]:
import pandas as pd
import numpy as np
import math

In [14]:
filepath = 'watermelon3_0alpha.csv'

In [21]:
df = pd.read_csv(filepath)
df
#print(df['density'] > 0.26)
# ind = np.random.randint(17,size = 17)
#print(ind)
# df.loc[ind]

,id,density,suger_ratio,clas
0,1,0.697,0.460,1
1,2,0.774,0.376,1
2,3,0.634,0.264,1
3,4,0.608,0.318,1
4,5,0.556,0.215,1
5,6,0.403,0.237,1
6,7,0.481,0.149,1
7,8,0.437,0.211,1
8,9,0.666,0.091,-1
9,10,0.243,0.267,-1


In [15]:
"""
input:
    dataframe:样本集
    class_atrr:决策属性
output:
    ShannonEntropy
"""

def Ent(dataframe, clas_atrr):
    row, col = dataframe.shape
    grouped_atrr = dataframe.groupby(by = clas_atrr).groups
    p = []
    for key in grouped_atrr:
        p.append(len(grouped_atrr[key])/row)
    ent = 0
    for value in p:
        ent += -(value * math.log(value, 2)) 
#         print(ent)
    return ent

In [16]:
"""
input:
    dataframe:样本集
    连续属性atrr:对属性atrr对样本集进行划分所获得的 "信息增益"
output:
    属性atrr对样本集进行划分所获得的信息增益
"""
def GainCont(dataframe, atrr, clas_atrr = 'clas'):
    row, col = dataframe.shape
    data_atrr = df[atrr]
    values = list(set(data_atrr)); values.sort()
    Ta = [(values[i]+values[i+1])/2 for i in range(len(values)-1)]
    ent_t_all = []
    for t in Ta:
        group_pos = dataframe[dataframe[atrr] >= t]
        group_neg = dataframe[dataframe[atrr] < t ]
#         grouped_pos = group_pos.groupby(by = clas_atrr).groups
#         grouped_neg = group_neg.groupby(by = clas_atrr).groups
        ent_t = (len(group_pos)/row)*Ent(group_pos, clas_atrr) + (len(group_neg)/row*Ent(group_neg, clas_atrr))
        ent_t_all.append(ent_t)
    ent_d = Ent(dataframe, clas_atrr)   
    gain_all = [ent_d-v for v in ent_t_all]
    gain_t = max(gain_all)
    ind = gain_all.index(gain_t)
    
    return gain_t, Ta[ind]

In [17]:
"""
input:
    dataframe:样本集
    A:属性集
    clas_atrr:决策属性
output:
    以node 为根节点的一棵决策树
"""
def TreeGenerate(dataframe, A, clas_atrr, df):
#     decisionTree = {}
    #groupedA = dataframe.groupby(by = A).groups
    grouped = dataframe.groupby(by = clas_atrr).groups
#     clas = dataframe[clas_atrr]; del dataframe[clas_atrr]
    if len(grouped) == 1:
        return list(grouped.keys())[0]
    if len(A) == 0 :
        print(list(grouped.keys()))
        list_g = list(grouped.values())
#         print(list_g)
        list_num = [len(v) for v in list_g]
        return list(grouped.keys())[list_num.index(max(list_num))]
    groupedA = dataframe.groupby(by = A).groups
    if len(groupedA) == 1:
        return list(grouped.keys())[0]
    gain_atrr = []
    sep_atrr = []
#     for atrr in A:
#         if atrr == '密度' or atrr == '含糖量'：
#             gaincont,sep = GainCont(dataframe, atrr, clas_atrr)
#             gain_atrr.append(gaincont)
#         else:
#             gain_atrr.append(Gain(dataframe, atrr, clas_atrr))
    for atrr in A:
        gain_temp, sep_temp = GainCont(dataframe, atrr)
        gain_atrr.append(gain_temp)
        sep_atrr.append(sep_temp)
    ind_best = gain_atrr.index(max(gain_atrr))
    sep_best = sep_atrr[ind_best]
    a_best = A[ind_best]
    keyword = a_best + ':' + str(sep_best)
    decisionTree = {keyword:{}}; del A[ind_best]
#     dataframe[clas_atrr] = clas
    group_pos = dataframe[dataframe[a_best] >= sep_best]
    group_neg = dataframe[dataframe[a_best] < sep_best ]
    for key in ['group_pos', 'group_neg']:
        if key == 'group_pos':
            #print('pos')
            if len(group_pos) == 0:
                list_g = list(grouped.values())
                list_num = [len(v) for v in list_g]  
                decisionTree[keyword][key] = list(grouped.keys())[list_num.index(max(list_num))]
                continue
            else:
                subA = A[:]
                #print(subA)
                dfv = group_pos
                #print(dfv)
                decisionTree[keyword][key] = TreeGenerate(dfv, subA, clas_atrr, df)
        if key == 'group_neg':
            #print('neg')
            if len(group_neg) == 0:
                list_g = list(grouped.values())
                list_num = [len(v) for v in list_g]  
                decisionTree[keyword][key] = list(grouped.keys())[list_num.index(max(list_num))]
                continue
            else:
                subA = A[:]
                #print(subA)
                dfv = group_neg
                #print(dfv)
                decisionTree[keyword][key] = TreeGenerate(dfv, subA, clas_atrr, df)
    return decisionTree

In [39]:
"""
input:
    dataframe:样本集
    T:训练轮数
output:
    分类任务投票法;回归任务简单平均法
    H(x)
"""
def Bagging(dataframe, T):
    row,_ = dataframe.shape
    A = list(df);A.remove('id');A.remove('clas')
    classifier = []
    for i in range(T):
        Ac = A.copy() 
        ind = np.random.randint(row, size = row)
        df_bs = dataframe.loc[ind]
#         print(df_bs)
        h = TreeGenerate(df_bs, Ac, 'clas', df_bs)
        classifier.append(h)
    
    return classifier

In [29]:
def classify(tree, A, x):
    firstkey = list(tree.keys())[0]
    secondDict = tree[firstkey]
    atrr_ind = A.index(firstkey)
    for key in secondDict.keys():
        if x[atrr_ind] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify_by_decisionTree(secondDict[key], A, x)
            else:
                classLabel = secondDict[key]
    
    return classLabel

In [40]:
classifier = Bagging(df, 3)

    id  density  suger_ratio  clas
14  15    0.360        0.370    -1
6    7    0.481        0.149     1
11  12    0.343        0.099    -1
13  14    0.657        0.198    -1
6    7    0.481        0.149     1
1    2    0.774        0.376     1
3    4    0.608        0.318     1
5    6    0.403        0.237     1
14  15    0.360        0.370    -1
0    1    0.697        0.460     1
6    7    0.481        0.149     1
9   10    0.243        0.267    -1
12  13    0.639        0.161    -1
7    8    0.437        0.211     1
16  17    0.719        0.103    -1
11  12    0.343        0.099    -1
10  11    0.245        0.057    -1
[-1, 1]
    id  density  suger_ratio  clas
14  15    0.360        0.370    -1
15  16    0.593        0.042    -1
16  17    0.719        0.103    -1
13  14    0.657        0.198    -1
8    9    0.666        0.091    -1
11  12    0.343        0.099    -1
16  17    0.719        0.103    -1
7    8    0.437        0.211     1
1    2    0.774        0.376     1
1    2    0.

In [24]:
import json

In [41]:
json.dumps(classifier)

'[{"density:0.38149999999999995": {"group_pos": {"suger_ratio:0.20450000000000002": {"group_pos": 1, "group_neg": -1}}, "group_neg": -1}}, {"suger_ratio:0.373": {"group_pos": 1, "group_neg": {"density:0.4590000000000001": {"group_pos": -1, "group_neg": -1}}}}, {"suger_ratio:0.20450000000000002": {"group_pos": 1, "group_neg": -1}}]'

In [42]:
t = classifier[0]

In [43]:
json.dumps(t)

'{"density:0.38149999999999995": {"group_pos": {"suger_ratio:0.20450000000000002": {"group_pos": 1, "group_neg": -1}}, "group_neg": -1}}'

In [45]:
t.keys()

dict_keys(['density:0.38149999999999995'])